For this work, KFUPM & King Abdulaziz Center for World Culture has been selected.

In [ ]:
%pip install -q osmnx folium networkx shapely pyproj

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import osmnx as ox
import networkx as nx
import folium
import time
import random
import math

In [26]:
# Coordinates: (lat, lon)
kfupm = (26.3066, 50.1450)
ithra = (26.3278, 50.1545)

# Map center
center = ((kfupm[0] + ithra[0]) / 2,
          (kfupm[1] + ithra[1]) / 2)

print("KFUPM:", kfupm)
print("Ithra:", ithra)
print("Center:", center)
def base_map(center, kfupm, ithra):
    m = folium.Map(location=center, zoom_start=14)
    folium.Marker(kfupm, popup="KFUPM", icon=folium.Icon(color="blue")).add_to(m)
    folium.Marker(ithra, popup="Ithra", icon=folium.Icon(color="green")).add_to(m)
    return m

m0 = base_map(center, kfupm, ithra)
m0

KFUPM: (26.3066, 50.145)
Ithra: (26.3278, 50.1545)
Center: (26.3172, 50.14975)


Graph

In [27]:
ox.settings.use_cache = True
ox.settings.log_console = True
ox.settings.timeout = 180

G = ox.graph_from_point(center, dist=3000, network_type="drive", simplify=True)

print("Nodes:", len(G.nodes))
print("Edges:", len(G.edges))

Nodes: 2674
Edges: 6530


Nearest Nodes & Helper Functions

In [28]:
src = ox.distance.nearest_nodes(G, X=kfupm[1], Y=kfupm[0])
dst = ox.distance.nearest_nodes(G, X=ithra[1], Y=ithra[0])

print("Source node:", src)
print("Destination node:", dst)
def path_length_m(G, path):
    """Total path length in meters, using edge 'length'."""
    total = 0.0
    for u, v in zip(path[:-1], path[1:]):
        data = G.get_edge_data(u, v)
        best = min(attr.get("length", float("inf")) for attr in data.values())
        total += best
    return total

def add_route_to_map(m, G, path, name, color=None):
    """Add route to map."""
    coords = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in path]
    folium.PolyLine(coords, tooltip=name, weight=5, color=color).add_to(m)
    return m

Source node: 6582857339
Destination node: 1489746234


BFS

In [29]:
start = time.perf_counter()
bfs_path = nx.shortest_path(G, source=src, target=dst)
bfs_time = time.perf_counter() - start
bfs_dist = path_length_m(G, bfs_path)

print(f"BFS time: {bfs_time:.6f} s")
print(f"BFS distance: {bfs_dist:.0f} m")

m_bfs = base_map(center, kfupm, ithra)
m_bfs = add_route_to_map(m_bfs, G, bfs_path, "BFS", color="blue")
m_bfs

BFS time: 0.000722 s
BFS distance: 6492 m


DFS

In [30]:
def dfs_path_find(G, source, target):
    stack = [(source, [source])]
    visited = set()

    while stack:
        node, path = stack.pop()
        if node == target:
            return path
        if node in visited:
            continue
        visited.add(node)

        for nbr in G.successors(node):
            if nbr not in visited:
                stack.append((nbr, path + [nbr]))

    raise nx.NetworkXNoPath("DFS did not find a path")

start = time.perf_counter()
dfs_path = dfs_path_find(G, src, dst)
dfs_time = time.perf_counter() - start
dfs_dist = path_length_m(G, dfs_path)

print(f"DFS time: {dfs_time:.6f} s")
print(f"DFS distance: {dfs_dist:.0f} m")

m_dfs = base_map(center, kfupm, ithra)
m_dfs = add_route_to_map(m_dfs, G, dfs_path, "DFS", color="red")
m_dfs

DFS time: 0.012986 s
DFS distance: 73468 m


Dijkstra

In [31]:
start = time.perf_counter()
dijkstra_path = nx.shortest_path(G, source=src, target=dst, weight="length", method="dijkstra")
dijkstra_time = time.perf_counter() - start
dijkstra_dist = path_length_m(G, dijkstra_path)

print(f"Dijkstra time: {dijkstra_time:.6f} s")
print(f"Dijkstra distance: {dijkstra_dist:.0f} m")

m_dij = base_map(center, kfupm, ithra)
m_dij = add_route_to_map(m_dij, G, dijkstra_path, "Dijkstra", color="green")
m_dij

Dijkstra time: 0.010414 s
Dijkstra distance: 6243 m


Simulated Annealing
Max. Iterations: 250, Starting Temp.: 1, Geometric Cooling with alpha = 0.995

In [35]:
def random_dfs_path(G, source, target):
    stack = [(source, [source])]
    visited = set()

    while stack:
        node, path = stack.pop()

        if node == target:
            return path

        if node in visited:
            continue

        visited.add(node)

        neighbors = list(G.successors(node))
        random.shuffle(neighbors)

        for nbr in neighbors:
            if nbr not in visited:
                stack.append((nbr, path + [nbr]))

    raise nx.NetworkXNoPath("No path found")


def simulated_annealing_route(G, source, target,
                              iters=400,
                              T0=10.0,
                              alpha=0.995,
                              seed=7):

    rnd = random.Random(seed)

    current = random_dfs_path(G, source, target)
    current_cost = path_length_m(G, current)

    best = current
    best_cost = current_cost

    T = T0

    for _ in range(iters):

        if len(current) < 6:
            break

        i = rnd.randint(1, len(current) - 4)
        j = rnd.randint(i + 2, len(current) - 2)

        a = current[i]
        b = current[j]

        try:
            mid = nx.shortest_path(G, source=a, target=b, weight="length")
        except nx.NetworkXNoPath:
            T *= alpha
            continue

        candidate = current[:i] + mid + current[j+1:]
        cand_cost = path_length_m(G, candidate)

        delta = cand_cost - current_cost

        if delta < 0 or rnd.random() < math.exp(-delta / T):
            current = candidate
            current_cost = cand_cost

            if cand_cost < best_cost:
                best = candidate
                best_cost = cand_cost

        T *= alpha

    return best

start = time.perf_counter()
sa_path = simulated_annealing_route(G, src, dst,
                                    iters=400,
                                    T0=10.0,
                                    alpha=0.995,
                                    seed=7)

sa_time = time.perf_counter() - start
sa_dist = path_length_m(G, sa_path)

print(f"SA time: {sa_time:.6f} s")
print(f"SA distance: {sa_dist:.0f} m")

m_sa = base_map(center, kfupm, ithra)
m_sa = add_route_to_map(m_sa, G, sa_path, "Simulated Annealing", color="purple")
m_sa

SA time: 0.734791 s
SA distance: 6243 m


Final comparison (Time & Distance)

In [37]:
results = [
    ("BFS", bfs_time, bfs_dist, len(bfs_path)),
    ("DFS", dfs_time, dfs_dist, len(dfs_path)),
    ("Dijkstra", dijkstra_time, dijkstra_dist, len(dijkstra_path)),
    ("Simulated Annealing", sa_time, sa_dist, len(sa_path)),
]

print(f"{'Algorithm':<22} {'Time(s)':>10} {'Distance(m)':>14} {'Nodes in Path':>14}")
print("-" * 64)
for name, t, d, n_nodes in results:
    print(f"{name:<22} {t:>10.6f} {d:>14.0f} {n_nodes:>14}")

Algorithm                 Time(s)    Distance(m)  Nodes in Path
----------------------------------------------------------------
BFS                      0.000722           6492             26
DFS                      0.012986          73468            475
Dijkstra                 0.010414           6243             36
Simulated Annealing      0.734791           6243             36


#Conclusion
1. BFS was the fastest algorithm (0.0007 s), but it does not consider edge weights. It produced a slightly longer route (6492 m) compared to the optimal solution.
2. DFS performed poorly for routing purposes.
3. Dijkstra produced the shortest route (6243 m) in a reasonable time.
4. Simulated Annealing converged to the same optimal distance as Dijkstra, but it required significantly more computation time (0.73 s), making it less efficient for small graph problems.